In [1]:
import numpy as np
import re
from pathlib import Path
import numpy as np
import pandas as pd
import json


fname = Path("../results/logs_nim/L1000_nim_10.txt")
text = fname.read_text(encoding="utf-8")
tokens = re.findall(r'\d+(?:\.\d+)?it/s', text)
rates_with_unit = tokens
rates = [float(t.replace("it/s", "")) for t in tokens]
log_dir = Path("../results/logs_nim")
files = sorted(log_dir.glob("*nim*.txt"))
pattern = re.compile(r'(\d+(?:\.\d+)?)it/s')
rows = {}
results = []

def safe_mean(arr):
    return float(np.nan) if len(arr)==0 else float(np.mean(arr))

for fp in files:
    text = fp.read_text(encoding="utf-8", errors="ignore")
    matches = pattern.findall(text)
    vals = [float(m.replace(",", ".")) for m in matches]
    rows[fp.name] = vals
    group1 = [v for i, v in enumerate(vals) if (i % 4) in (0, 1)]
    group2 = [v for i, v in enumerate(vals) if (i % 4) in (2, 3)]
    row = {
        "file": fp.name,
        "mean_group1": safe_mean(group1),
        "mean_group2": safe_mean(group2)
    }
    results.append(row)

df = pd.DataFrame(results).sort_values("file").reset_index(drop=True)
out_csv = log_dir / "rates_summary_grouped.csv"
df.to_csv(out_csv, index=False)
out_json = log_dir / "rates_per_file_grouped.json"
with open(out_json, "w", encoding="utf-8") as f:
    json.dump({"values": rows}, f, indent=2, ensure_ascii=False)

df['time'] = df.apply(lambda x: (16/x.mean_group1 + 16/x.mean_group2), axis=1)
df['mean_group1'] = df.apply(lambda x: 16/x.mean_group1, axis=1)
df['mean_group2'] = df.apply(lambda x: 16/x.mean_group2, axis=1)
df = df[['file', 'mean_group1', 'mean_group2', 'time']]
df.to_csv('time_nim.tsv', index=None, sep='\t')
